In [1]:
import math
from scipy.stats import norm
from scipy.optimize import fsolve

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
S0 = 56
K = 60
r = 0.0450
q = 0.015
T = 8/12
Vm = 6.95

In [4]:
def blackScholesVanilla(K , T , S , v , q , r , CP ):
    phi = 1 if CP == 'C' else -1
    dp = (1/(v*math.sqrt(T)))*(math.log(S/K) + ((r - q + (v**2)/2)*T) )
    dm = dp - (v*math.sqrt(T))
    vBS = phi*((norm.cdf(phi*dp)*(S*math.exp(-q*T))) - ((norm.cdf(phi*dm))*(K*math.exp(-r*T))))
    return vBS

In [5]:
def calculateIV(v , K , T , S , q , r , CP , Vm):
    
    modelV = blackScholesVanilla(K , T , S , v , q , r , CP )
    eq = modelV - Vm
    return eq

In [6]:
IV = fsolve(calculateIV , x0 = 0.25 , args =( K , T , S0 , q, r , 'P', Vm))

In [7]:
print(IV)

[0.29489447]


In [8]:
def trinomialTreePricer(S , K , v, r , q , T , N):
    deltaT = T/N
    u = math.exp(v*math.sqrt(3*deltaT))
    d = 1/u
    m = 1
    #f = math.exp((r-q)*(deltaT))
    df = math.exp(-r*deltaT)
    t1 = (r-q-((v**2)/2))
    t2 = math.sqrt(deltaT/(12*(v**2)))
    pU = (1/6) + (t1*t2) 
    pD = (1/6) - (t1*t2)
    pM = (2/3)
    V_N = []
    for i in range((2*N)+1):
        V_N.append(max(K - S*(u**(N-i)),0))
    
    for j in range(N-1,-1,-1):
        V = []
        for i in range((2*j) +1):
            p = df*((pU*V_N[i]) + (pM*V_N[i+1]) + (pD*(V_N[i+2])))
            #American Pricer
            am = max(K - S*(u**(j-i)) , 0)
            p = max(p,am)
            V.append(p)
        V_N = V
    
    return V_N[0]

In [9]:
def trinomialTreeBSPricer(S , K , v, r , q , T , N):
    deltaT = T/N
    u = math.exp(v*math.sqrt(3*deltaT))
    d = 1/u
    m = 1
    #f = math.exp((r-q)*(deltaT))
    df = math.exp(-r*deltaT)
    t1 = (r-q-((v**2)/2))
    t2 = math.sqrt(deltaT/(12*(v**2)))
    pU = (1/6) + (t1*t2) 
    pD = (1/6) - (t1*t2)
    pM = (2/3)
    V_N = []
    for i in range((2*N)-1):
        V_N.append(blackScholesVanilla(K , deltaT , S*(u**(N-1-i)) , v , q , r , 'P' ))
    
    for j in range(N-2,-1,-1):
        V = []
        for i in range((2*j) +1):
            p = df*((pU*V_N[i]) + (pM*V_N[i+1]) + (pD*(V_N[i+2]))) 
            #American Pricer
            am = max(K - S*(u**(j-i)) , 0)
            p = max(p,am)
            V.append(p)
        V_N = V
    
    return V_N[0]

In [10]:
def trinomialTreeBSRPricer(S , K , v, r , q , T , N):
    t1 = trinomialTreeBSPricer(S , K , v, r , q , T , 2*N)
    t2 = trinomialTreeBSPricer(S , K , v, r , q , T , N)
    
    return (2*t1) - t2

In [12]:

N = 10
tol = 1e-6
v = 0.25
prevV = trinomialTreeBSRPricer(S0 , K , v , r , q , T , N)
N = 2*N
currV = trinomialTreeBSRPricer(S0 , K , v , r , q , T , N)

print(prevV ,currV)
while abs(currV - prevV) > tol:
    
    prevV = currV
    N = 2*N
    currV = trinomialTreeBSRPricer(S0 , K , v , r , q , T , N)
    
    print(N , prevV,currV)

print(N , currV)

6.351300492928197 6.350992986889087
40 6.350992986889087 6.352356153139095
80 6.352356153139095 6.350598610596669
160 6.350598610596669 6.3509048495649765
320 6.3509048495649765 6.350884443634775
640 6.350884443634775 6.350845185096149
1280 6.350845185096149 6.350838349906355
2560 6.350838349906355 6.35084050109749
5120 6.35084050109749 6.350839576930721
5120 6.350839576930721


In [13]:
N_fixed = N

In [14]:
import numpy as np

def calculate_ABC(y, R):
    A = ((np.exp(((1 - (2 / np.pi)) * y)) - np.exp(-((1 - (2 / np.pi)) * y))) ** 2)
    B = (4 * (np.exp(((2/np.pi) * y)) + np.exp(-(((2 / np.pi) * y)))) - (2 * np.exp(-y) * ((np.exp((1 - (2 / np.pi)) * y) + np.exp(-((1 - (2 / np.pi)) * y))) * (np.exp((2 * y)) + 1) - (R ** 2))))
    C = (np.exp((-2) * y) * ((R ** 2 - ((np.exp(y) - 1) ** 2)) * ((np.exp(y) + 1) ** 2) - R**2))
    print(A,B,C)
    return A, B, C

def calculate_ABC(y, R):
    # Compute terms for A
    term1_A = math.exp((1 - (2/math.pi)) * y)
    term2_A = math.exp(-(1 - (2/math.pi)) * y)
    A = (term1_A - term2_A) ** 2

    # Compute terms for B
    term1_B = 4 * (math.exp((2/math.pi) * y) + math.exp(-(2/math.pi) * y))
    term2_B = 2 * math.exp(-y) * (math.exp((1 - (2/math.pi)) * y) + math.exp(-(1 - (2/math.pi)) * y))
    term3_B = (math.exp(2 * y) + 1 - (R ** 2))
    B = term1_B - (term2_B * term3_B)

    # Compute terms for C
    term1_C = math.exp(-2 * y)
    term2_C = (R ** 2) - ((math.exp(y) - 1) ** 2)
    term3_C = ((math.exp(y) + 1) ** 2) - (R ** 2)
    C = term1_C * term2_C * term3_C

    return A, B, C

def A_f(x):
    common_term = 0.5 * np.sqrt(1 - np.exp(-2 * x**2) / np.pi)
    if x >= 0:
        return 0.5 + common_term
    else:
        return 0.5 - common_term

def approx_implied_vol_call(Cm, K, T, F, r):
    y = np.log(F / K)
    alphaC = Cm/(K*np.exp(-r * T))
    R = ((2 * alphaC) - np.exp(y) + 1)
    A, B, C = calculate_ABC(y, R)
    beta = ((2 * C) / (B + np.sqrt((B ** 2) + (4 * A * C))))
    gamma = (-(np.pi / 2) * np.log(beta))
    if y >= 0:
        C0 = K * np.exp(-r * T) * ((np.exp(y) * (A_f(np.sqrt((2 * y))))) - 0.5)
        if Cm <= C0:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) - np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    else:
        C0 = K * np.exp(-r * T) * ((np.exp(y) / 2) - A_f(-np.sqrt((-2) * y)))
        if Cm <= C0:
            sigma = (1 / np.sqrt(T)) * ((-np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    return sigma

def approx_implied_vol_put(Pm, K, T, F, r):
    print(Pm, K, T, F, r)
    y = np.log(F / K)
    alphaP = Pm/(K*np.exp(-r * T))
    R = ((2 * alphaP) + np.exp(y) - 1)
    A, B, C = calculate_ABC(y, R)
    print(A,B,C)
    beta = ((2 * C) / (B + np.sqrt((B ** 2) + (4 * A * C))))
    gamma = (-(np.pi / 2) * np.log(beta))
    print(beta , gamma)
    if y >= 0:
        P0 = K * np.exp(-r * T) * (0.5 - (np.exp(y) * A_f(-np.sqrt((2 * y)))))
        print(y,P0)
        if Pm <= P0:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) - np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    else:
        P0 = K * np.exp(-r * T) * ((A_f(np.sqrt((-2) * y))) - ((np.exp(y) / 2)))
        if Pm <= P0:
            sigma = (1 / np.sqrt(T)) * ((-np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
        else:
            sigma = (1 / np.sqrt(T)) * ((np.sqrt(gamma + y)) + np.sqrt((gamma - y)))
    return sigma


In [15]:
F = S0*math.exp((r-q)*T)
iv_approx = approx_implied_vol_put(Vm, K, T, F, r)

6.95 60 0.6666666666666666 57.13127504149833 0.045
0.0012679269932118766 0.14615001214802703 0.14221647939920812
0.9650066782055413 0.05595216924283666


In [16]:
iv_approx

0.29458765639080964

In [18]:
#Secant Implementation
iv0 = iv_approx - 0.02

iv1 = iv_approx + 0.02
tol = 1e-4
while abs(iv0 - iv1) > tol:
    f1 = trinomialTreeBSRPricer(S0 , K , iv1 , r , q , T , N_fixed)
    f0 = trinomialTreeBSRPricer(S0 , K , iv0 , r , q , T , N_fixed)
    print(iv0 , f0 , iv1 , f1)
    iv2 = iv1 - (((f1-Vm)*(iv1-iv0))/(f1 - f0))
    iv0 = iv1
    iv1 = iv2
    
print(iv1 , trinomialTreeBSRPricer(S0 , K , iv1 , r , q , T , N_fixed))

0.2745876563908096 6.782130131531267 0.31458765639080966 7.491157526067803
0.31458765639080966 7.491157526067803 0.28405808699827373 6.9493052115006675
0.2840972334467307 6.949997256410611
